# Add Github Issues to Qdrant Vector Database

In [ ]:
!pip install -U qdrant-client sentence-transformers

In [1]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

/Users/alokdwivedi/dev/github/dashamlav/llm/langchain/lang_chain/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

In [3]:
import os
import dotenv

QDRANT_COLLECTION_NAME="github_issues_coll"
OPEN_AI_MODEL_NAME="gpt-4o-mini" # "gpt-3.5-turbo-16k"
CONFIG_FILE_PATH = '.7ytrepmnt'
dotenv.load_dotenv(CONFIG_FILE_PATH)
print(os.getenv("QDRANT_HOST_URL"))

localhost:6333


In [5]:
# https://stackoverflow.com/questions/78399709/limit-context-token-on-document-retrieval-chains

from qdrant_client import QdrantClient
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from qdrant_client.http.models import Distance, VectorParams

def get_qdrant_client():
        qdrant_url = os.getenv("QDRANT_HOST_URL")
        qdrant_api_key = os.getenv("QDRANT_API_KEY")
        
        return QdrantClient(
            url=qdrant_url, 
            api_key=qdrant_api_key,
        )

def get_existing_vector_store(collection_identity):
        qdrant_client = get_qdrant_client()

        embeddings=OpenAIEmbeddings()
        return Qdrant(
            client=qdrant_client,
            collection_name=collection_identity,
            embeddings=embeddings,
        )

def get_chunked_text(file_name):
    with open(file_name, "r", encoding="utf-8") as f:
        text = f.read()
        text_splitter = CharacterTextSplitter(
            separator='\n',
            chunk_size=1000,
            chunk_overlap=150,
            length_function=len
        )
        return text_splitter.split_text(text)

In [6]:
# https://www.brainbyte.io/vector-search-using-openai-embeddings-with-qdrant/
get_qdrant_client().create_collection(
    collection_name=QDRANT_COLLECTION_NAME,
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

True

In [7]:
chunked_text_temp = get_chunked_text(f"gh_issues/github_issues_1.txt")

In [8]:
len(chunked_text_temp)

129

In [ ]:

qdrant_collection = get_existing_vector_store(QDRANT_COLLECTION_NAME)
for page_num in range(19):
    chunked_text = get_chunked_text(f"gh_issues/github_issues_{page_num}.txt")
    vector_ids = qdrant_collection.add_texts(chunked_text)

In [10]:
query = "Facing issues with common ForkJoinPool used in ExecutableJar"
found_docs = qdrant_collection.similarity_search(query)

In [11]:
print(f"Total docs found:{len(found_docs)}")
print(found_docs[0].page_content)


Total docs found:4
***End of Issue number:39928***
***Begining of Issue number:39843***
Issue Title:Incorrect classloader used by common ForkJoinPool when using Executable Jar
Issue Body:# Context
I created this issue as a bug report or enhancement proposal - depending on how would you classify current behaviour.
I have a spring application that I am building using "org.springframework.boot gradle" plugin. This plugin builds Executable Jar and War as described in documentation:
https://docs.spring.io/spring-boot/docs/current/reference/html/executable-jar.html
# Problem
Executable Jar uses custom class loader: `org.springframework.boot.loader.launch.LaunchedClassLoader` when running the application. 
This class loader is not propagated to the common ForkJoinPool, which uses system class loader by default.
Take a code like that:
```
IntStream.rangeClosed(0, 4)
    .parallel()
    .forEach(i -> System.out.println(Thread.currentThread().getName() + " " + Thread.currentThread().getContextCl

## Ingest some Closed issues as well

In [ ]:
qdrant_collection = get_existing_vector_store(QDRANT_COLLECTION_NAME)
for page_num in range(20):
    chunked_text = get_chunked_text(f"gh_issues/closed/github_issues_{page_num}.txt")
    vector_ids = qdrant_collection.add_texts(chunked_text)

In [12]:
query = "Facing issues with Micrometer Tracing and OpenTelemetry"
found_docs = qdrant_collection.similarity_search(query)
print(f"Total docs found:{len(found_docs)}")
print(found_docs[0].page_content)


Total docs found:4
Comments:
***End of Issue number:43202***
***Begining of Issue number:43200***
Issue Title:Spring Boot 3.3.x dependencies do not converge for Micrometer Tracing and OpenTelemetry
Issue Body:Found that `org.springframework.boot:spring-boot-dependencies:3.3.5` specifies `opentelemetry-bom` version `1.37.0` and `micrometer-tracing-bom` version `1.3.5`, but that version of micrometer-tracing depends on opentelemetry `1.38.0` dependencies.
This can be worked around by overloading the version of `opentelemetry-bom` to be version `1.38.0` in the Maven `<dependencyManagement>` section.
